---

We follow the same pipeline

---

training pipeline in pytorch
* Design model(input,output size, forward pass)
* Construct loss and optimizer
* Training loop
    * forward pass: compute prediction
    * backward pass: gradients
    * update weights

We are here doing binary classification:
equations are as follows:

$ z = x.w+b $, this is same as the linear regression layer\
$ \hat{y} = σ(z) $\
where, $σ = \frac{1}{1+e^{-z}}$

In [20]:
import torch
import torch.nn as nn
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import numpy as np

# prepare data
bc = datasets.load_breast_cancer()
x,y = bc.data, bc.target
n_samples, n_features = x.shape
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size = 0.2,random_state = 1234)

#scaling
n_features
sc = StandardScaler()
x_train = sc.fit_transform(x_train) # mean = 0 & sd = 1
x_test = sc.transform(x_test)
x_train = torch.from_numpy(x_train.astype(np.float32))
x_test = torch.from_numpy(x_test.astype(np.float32))
y_train = torch.from_numpy(y_train.astype(np.float32))
y_test = torch.from_numpy(y_test.astype(np.float32))
y_train = y_train.view(y_train.shape[0],1)
y_test = y_test.view(y_test.shape[0],1)

# model
class LogisticRegression(nn.Module):
    def __init__(self,n_input_features):
        super(LogisticRegression, self).__init__()
        self.linear = nn.Linear(n_input_features,1)
    def forward(self,x):
        y_predicted = torch.sigmoid(self.linear(x))
        return y_predicted

model = LogisticRegression(n_features)

# loss and optimizer
criterion = nn.BCELoss()
optimizer = torch.optim.NAdam(model.parameters(),lr = 0.01)

# training loop
num_epoch = 100
for epoch in range(num_epoch):

    # forward pass
    y_predicted = model(x_train)

    # loss
    loss = criterion(y_predicted, y_train)

    # backward pass
    loss.backward()

    # update parameters
    optimizer.step()

    optimizer.zero_grad()

    if (epoch+1)%10==0:
        print(f'epoch:{epoch+1}, loss = {loss.item():.4f}')

with torch.no_grad():
    y_predicted = model(x_test)
    y_predicted_class = y_predicted.round()

    #accuracy: how many classifications are correct among the total number of classification
    acc = y_predicted_class.eq(y_test).sum() / float(y_test.shape[0])
    print(f'accuracy = {acc.item():.4f}')




epoch:10, loss = 0.3733
epoch:20, loss = 0.2366
epoch:30, loss = 0.1752
epoch:40, loss = 0.1431
epoch:50, loss = 0.1239
epoch:60, loss = 0.1109
epoch:70, loss = 0.1013
epoch:80, loss = 0.0939
epoch:90, loss = 0.0880
epoch:100, loss = 0.0830
accuracy = 0.9298
